In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor
import optimizer as optim
import numpy as np
import random

ModuleNotFoundError: ignored

In [ ]:
def setup_seed(seed):
     torch.manual_seed(seed)
     torch.cuda.manual_seed_all(seed)
     np.random.seed(seed)
     random.seed(seed)
     torch.backends.cudnn.deterministic = True

setup_seed(20)

# Data, Model, Training, Valid

## Data

In [5]:
# Download training data from open datasets.
training_data = datasets.MNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = datasets.MNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

100%|██████████| 9912422/9912422 [00:00<00:00, 353724454.80it/s]

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 62344670.01it/s]


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 150348733.59it/s]

Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 24773119.33it/s]


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw



## Model

In [2]:
# Get cpu, gpu or mps device for training.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "cpu"
)
print(f"Using {device} device")

# Define model
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

Using cuda device


|    | torch optimizer acc  | custom optimizer acc |
  ----  | ----  | --- |
| SGD  | 72.4% | 72.4% |
| Momentum SGD  | 89.0% | 87.2% |
| Nestrov SGD  | 90.1% | 90.1% |
| Adam | 97.3% | 97.7% |
| Nadam | 97.1% | 97.4% |
| Adamw | 97.3% | 95.2% #(some bug here maybe) |

## Train

In [3]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")


def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [6]:
loss_fn = nn.CrossEntropyLoss()
model = NeuralNetwork().to(device)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3, weight_decay=0)
print(model)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.298019  [   64/60000]
loss: 0.286682  [ 6464/60000]
loss: 0.202501  [12864/60000]
loss: 0.220391  [19264/60000]
loss: 0.151500  [25664/60000]
loss: 0.319747  [32064/60000]
loss: 0.109737  [38464/60000]
loss: 0.230567  [44864/60000]
loss: 0.323673  [51264/60000]
loss: 0.177040  [57664/60000]
Test Error: 
 Accuracy: 96.0%, Avg loss: 0.131311 

Epoch 2
-------------------------------
loss: 0.070882  [   64/60000]
loss: 0.072354  [ 6464/60000]
loss: 0.099273  [12864/60000]
loss: 0.084804  [19264/60000]
loss: 0.036386  [25664/60000]
loss: 0.150246  [32064/60000]
loss: 0.047423  [38464/60000]
loss: 0.143967  [44864/60000]
loss: 0.1

# Training with default optimizer

In [8]:
import importlib
import optimizer as optim
importlib.reload(optim)

<module 'optimizer' from '/content/optimizer.py'>

In [9]:
loss_fn = nn.CrossEntropyLoss()
model = NeuralNetwork().to(device)
# optimizer = optim.SGD(model.parameters(), lr=1e-3)
# optimizer = optim.Momentum(model.parameters(), lr=1e-3)
# optimizer = optim.Nestrov(model.parameters(), lr=1e-3)
# optimizer = optim.Adam(model.parameters(), lr=1e-3)
# optimizer = optim.Nadam(model.parameters(), lr=1e-3)
optimizer = optim.Adamw(model.parameters(), lr=1e-3)
print(model)

epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)
Epoch 1
-------------------------------
loss: 2.306578  [   64/60000]
loss: 0.434851  [ 6464/60000]
loss: 0.335383  [12864/60000]
loss: 0.333054  [19264/60000]
loss: 0.249722  [25664/60000]
loss: 0.323282  [32064/60000]
loss: 0.218301  [38464/60000]
loss: 0.352347  [44864/60000]
loss: 0.363521  [51264/60000]
loss: 0.284657  [57664/60000]
Test Error: 
 Accuracy: 93.1%, Avg loss: 0.231945 

Epoch 2
-------------------------------
loss: 0.173764  [   64/60000]
loss: 0.212814  [ 6464/60000]
loss: 0.147696  [12864/60000]
loss: 0.223761  [19264/60000]
loss: 0.168054  [25664/60000]
loss: 0.258198  [32064/60000]
loss: 0.151443  [38464/60000]
loss: 0.279695  [44864/60000]
loss: 0.2

## Note for Nadam

the implementation of nadam in this code referenced the paper "An overview of gradient descent optimization algorithms"